In [26]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence

df1 = pd.read_csv('spam.csv')
df1['class'] = df1['Category'].map({'spam': 1, 'ham': 0})
y = df1['class']
X = df1['Message']

df1


,Category,Message,class
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will ü b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0


In [27]:

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)


In [28]:
tk = Tokenizer()
tk.fit_on_texts(X_train)
word_dict = tk.index_word
X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)

maxlen = 80
X_train_pad = sequence.pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = sequence.pad_sequences(X_test_seq, maxlen=maxlen)

In [29]:

word_dict[0] = '<PAD>'
word_dict[1] = '<START>'
word_dict[2] = '<UNK>'
for el in X_train_pad[0]:
    print(word_dict[el], end=' ')

<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> reply <UNK> win £100 weekly where will the 2006 fifa world cup be held send stop <UNK> 87239 <UNK> end service 

In [31]:
RNNSpam.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 8)           63632     
                                                                 
 lstm_3 (LSTM)               (None, 16)                1600      
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 65249 (254.88 KB)
Trainable params: 65249 (254.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
RNNSpam.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
RNNSpam.fit(X_train_pad, y_train,batch_size=64,epochs=10,validation_data=(X_test_pad, y_test))

Epoch 1/10


70/70 [==============================] - 4s 29ms/step - loss: 0.4606 - accuracy: 0.8640 - val_loss: 0.3301 - val_accuracy: 0.8664
Epoch 2/10
70/70 [==============================] - 2s 26ms/step - loss: 0.2080 - accuracy: 0.9186 - val_loss: 0.1396 - val_accuracy: 0.9767
Epoch 3/10
70/70 [==============================] - 2s 26ms/step - loss: 0.1063 - accuracy: 0.9785 - val_loss: 0.0942 - val_accuracy: 0.9830
Epoch 4/10
70/70 [==============================] - 2s 27ms/step - loss: 0.0704 - accuracy: 0.9881 - val_loss: 0.0766 - val_accuracy: 0.9848
Epoch 5/10
70/70 [==============================] - 2s 30ms/step - loss: 0.0467 - accuracy: 0.9912 - val_loss: 0.0636 - val_accuracy: 0.9865
Epoch 6/10
70/70 [==============================] - 2s 30ms/step - loss: 0.0333 - accuracy: 0.9946 - val_loss: 0.0595 - val_accuracy: 0.9883
Epoch 7/10
70/70 [==============================] - 2s 28ms/step - loss: 0.0254 - accuracy: 0.9960 - val_loss: 0.0520 - val_accuracy: 0.9883
Epoch 8/10
70/70 [======

In [34]:
#Not Spam 
test = ['Happy Birthday Arjun, hope you have a great day']
msgsq = tk.texts_to_sequences(test)
msgpadded = pad_sequences(msgsq, maxlen=30)
tk.index_word
RNNSpam.predict(msgpadded)

1/1 [==============================] - 1s 524ms/step


array([[0.00178551]], dtype=float32)

In [35]:
#Spam
test1 = ['This is the 2nd time we have tried 2 contact u. U have won the Â£750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate.']
msgsq1 = tk.texts_to_sequences(test1)
msgpadded1 = pad_sequences(msgsq1, maxlen=100)
tk.index_word
RNNSpam.predict(msgpadded1)

1/1 [==============================] - 0s 162ms/step


array([[0.9767136]], dtype=float32)